In [1]:
import numpy as np
import pandas as pd
from itertools import combinations, chain
import scipy.special as sp
from ttm import TTM

In [2]:
cov_dfs={}
cov_df1 = pd.read_csv('../data/cov_mat_2023-04-14.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[1]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-04-28.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[2]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-05-12.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[3]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-05-26.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[4]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-06-09.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[5]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-06-23.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[6]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-07-07.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[7]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-07-21.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[8]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-08-04.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[9]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-08-18.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[10]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-09-01.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[11]=cov_df1
cov_df1 = pd.read_csv('../data/cov_mat_2023-09-15.csv', index_col=0).filter(regex='^[D|A]')
cov_dfs[12]=cov_df1

In [3]:
id2user={i:x for i,x in enumerate(list(cov_df1.index))}
user2id={x:i for i,x in enumerate(list(cov_df1.index))}
id2cov={i:x for i,x in enumerate(list(cov_df1.columns))}
cov2id={x:i for i,x in enumerate(list(cov_df1.columns))}

cov_dfs_={}
cov_dfs_[1]= cov_dfs[1] +cov_dfs[2]
cov_dfs_[2]= cov_dfs[3] +cov_dfs[4]
cov_dfs_[3]= cov_dfs[5] +cov_dfs[6]
cov_dfs_[4]= cov_dfs[7] +cov_dfs[8]
cov_dfs_[5]= cov_dfs[9] +cov_dfs[10]
cov_dfs_[6]= cov_dfs[11] +cov_dfs[12]

In [4]:
cov_counts=[]
for i in range(1,13):
    cov_mat = cov_dfs[i].applymap(lambda x: 1 if x!=0 else 0)
    cov_counts.append(list(cov_mat.values.sum(axis=0)))
    
cov_counts = np.array(cov_counts)

In [5]:
valid_cov=[]
for cov, lst in enumerate(cov_counts.T):
    nonzeros = ([i for i in lst if i>=2])
    if len(nonzeros)==len(lst):
        valid_cov.append(cov)
print(len(valid_cov))
valid_cov_names=[id2cov[i] for i in valid_cov]

1150


In [6]:
user_counts=[]
for i in range(1,13):
    cov_mat = cov_dfs[i].applymap(lambda x: 1 if x!=0 else 0)
    cov_mat = cov_mat.loc[:, cov_mat.columns.isin(valid_cov_names)]
    user_counts.append(list(cov_mat.values.sum(axis=1)))
    
user_counts = np.array(user_counts)

In [7]:
valid_user=[]
for user, lst in enumerate(user_counts.T):
    nonzeros = ([i for i in lst if i>=2])
    if len(nonzeros)==len(lst):
        valid_user.append(user)
print(len(valid_user))
valid_user_names=[id2user[i] for i in valid_user]

798


# 0

In [8]:
cov_df = cov_dfs_[1] + cov_dfs_[2] + cov_dfs_[3] + cov_dfs_[4] + cov_dfs_[5] + cov_dfs_[6]
#cov_df = cov_dfs[2]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat= cov_df.values

L = 1
U = cov_mat.shape[0]
V = cov_mat.shape[1]
K = 5
iters=30

prev_theta_tensor = np.ones((L, K, U))
prev_theta_tensor *= 1/K

prev_alpha_tensor = np.ones((L, K, U))
prev_alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))
prev_phi_tensor *= 1/V

prev_beta_tensor = np.ones((L, K, V))
prev_beta_tensor *= 1/L

id2user_={i:x for i,x in enumerate(list(cov_df.index))} ##798
user2id_={x:i for i,x in enumerate(list(cov_df.index))} ##
id2cov_={i:x for i,x in enumerate(list(cov_df.columns))}  ##
cov2id_={x:i for i,x in enumerate(list(cov_df.columns))}  ##


ottm0=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10, prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=prev_beta_tensor, alpha_tensor=prev_alpha_tensor)
ottm0.transform_D()
ottm0._gibbs()
ottm0.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [9]:
T=20
topic2topwords1={}
topic2prob1={}
for j in range(ottm0.K):
    topic2prob1[j]={}
    topic2topwords1[j]=[]
    for w in range((ottm0.V)):
        topic2prob1[j][id2cov_[w]] = ottm0.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob1[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords1[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm0.cur_theta.sum(axis=1)/798)
pd.DataFrame(topic2topwords1)

[0.44378082 0.19150838 0.12797022 0.13899893 0.09774165]


,0,1,2,3,4
0,A_黄渤,D_让-吕克·戈达尔,D_史蒂文·斯皮尔伯格,A_本·阿弗莱克,A_刘德华
1,A_秦昊,D_香特尔·阿克曼,A_布拉德·皮特,A_克里斯·帕拉特,A_梁朝伟
2,A_克里斯·帕拉特,A_蒂尔达·斯文顿,A_罗伯特·德尼罗,A_佐伊·索尔达娜,A_张国荣
3,A_李雪健,D_洪常秀,D_文斯·吉里根,A_米歇尔·罗德里格兹,D_王晶
4,A_吴京,D_埃里克·侯麦,D_阿尔弗雷德·希区柯克,A_凯伦·吉兰,A_黄秋生
5,A_范伟,D_侯孝贤,A_加里·奥德曼,A_柯特妮·考克斯,D_杜琪峰
6,A_米歇尔·罗德里格兹,D_韦斯·安德森,D_米歇尔·麦克拉伦,A_真田广之,A_周星驰
7,A_汤姆·克鲁斯,A_马特·达蒙,A_克里斯蒂安·贝尔,D_盖·里奇,A_任达华
8,A_马特·达蒙,D_三宅唱,A_布莱恩·科兰斯顿,A_哈里森·福特,A_梁家辉
9,D_詹姆斯·古恩,D_伍迪·艾伦,A_汤姆·克鲁斯,A_史蒂文·元,A_古天乐


In [11]:
from scipy.stats import entropy
global_cluster=np.zeros(ottm0.cur_theta.shape[1],dtype=int)
for u,prop in enumerate(ottm0.cur_theta.T):
    l = prop.argmax()
    global_cluster[u] =  l
print(len(global_cluster))
from collections import Counter
entropy(list({k:v/798 for k,v in Counter(list(global_cluster)).items()}.values()))


798


1.2693611953229849

# 1

In [12]:

cov_df = cov_dfs[1]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

L = 3
U = cov_mat.shape[0]
V = cov_mat.shape[1]

iters=30

prev_theta_tensor = np.ones((L, K, U))
prev_theta_tensor[2]= ottm0.cur_theta
prev_theta_tensor[1]= ottm0.cur_theta
prev_theta_tensor[0]= ottm0.cur_theta

alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))
prev_phi_tensor[2]= ottm0.cur_phi
prev_phi_tensor[1]= ottm0.cur_phi
prev_phi_tensor[0]= ottm0.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

id2user_={i:x for i,x in enumerate(list(cov_df.index))}
user2id_={x:i for i,x in enumerate(list(cov_df.index))}
id2cov_={i:x for i,x in enumerate(list(cov_df.columns))}
cov2id_={x:i for i,x in enumerate(list(cov_df.columns))}


ottm1=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10, prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm1.transform_D()
ottm1._gibbs()
ottm1.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [14]:
T=20
topic2topwords1={}
topic2prob1={}
for j in range(ottm1.K):
    topic2prob1[j]={}
    topic2topwords1[j]=[]
    for w in range((ottm1.V)):
        topic2prob1[j][id2cov_[w]] = ottm1.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob1[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords1[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm1.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords1)

[346.11926904 128.1829181  104.53811953 113.86318018 105.29651315]


,0,1,2,3,4
0,D_孔大山,A_张国荣,D_伍迪·艾伦,A_詹妮弗·安妮斯顿,A_张国荣
1,A_克里斯·帕拉特,D_孔大山,D_史蒂文·斯皮尔伯格,A_塔伦·埃哲顿,A_周星驰
2,A_米歇尔·罗德里格兹,D_伊丹十三,A_凯特·温丝莱特,A_史蒂文·元,A_刘德华
3,A_安雅·泰勒-乔伊,A_梁朝伟,D_昆汀·塔伦蒂诺,A_亚当·桑德勒,A_张曼玉
4,A_克里斯·派恩,D_侯孝贤,A_布拉德·皮特,A_薛景求,A_吴孟达
5,A_塔伦·埃哲顿,D_维姆·文德斯,A_塞缪尔·杰克逊,A_黄政民,A_周润发
6,A_成龙,D_杨德昌,A_莱昂纳多·迪卡普里奥,A_全度妍,D_孔大山
7,A_薛景求,D_洪常秀,A_摩根·弗里曼,A_梅拉尼·罗兰,A_吴君如
8,A_黄政民,D_埃里克·侯麦,A_西尔维斯特·史泰龙,A_马克·斯特朗,A_梁家辉
9,A_全度妍,D_让-吕克·戈达尔,A_金·凯瑞,D_程耳,A_吴镇宇


# 2  

In [16]:

prev_theta_tensor = np.ones((L, K, U))
prev_theta_tensor[2]= ottm0.cur_theta
prev_theta_tensor[1]= ottm0.cur_theta
prev_theta_tensor[0]= ottm1.cur_theta

alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))
prev_phi_tensor[2]= ottm0.cur_phi
prev_phi_tensor[1]= ottm0.cur_phi
prev_phi_tensor[0]= ottm1.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L


cov_df = cov_dfs[2]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values



ottm2=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=iters,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm2.transform_D()
ottm2._gibbs()
ottm2.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [283]:

T=20
topic2topwords2={}
topic2prob2={}
for j in range(ottm2.K):
    topic2prob2[j]={}
    topic2topwords2[j]=[]
    for w in range((ottm2.V)):
        topic2prob2[j][id2cov_[w]] = ottm2.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob2[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords2[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm2.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords2)

[301.69290784 156.99469663  97.13671774 128.6265898  113.54908799]


,0,1,2,3,4
0,A_倪虹洁,D_铃木清顺,D_伍迪·艾伦,A_安娜·德·阿玛斯,A_刘德华
1,D_陈正道,D_罗曼·波兰斯基,D_宫崎骏,A_克里斯·埃文斯,A_张国荣
2,A_秦昊,D_克日什托夫·基耶斯洛夫斯基,D_李翰祥,A_瑞安·雷诺兹,A_梁朝伟
3,A_范伟,D_李沧东,A_海伦娜·伯翰·卡特,A_佩德罗·帕斯卡,A_张曼玉
4,D_辛爽,D_克莱尔·德尼,A_伍迪·艾伦,D_塔伊加·维迪提,A_古天乐
5,A_吴京,D_埃里克·侯麦,D_史蒂文·斯皮尔伯格,A_塞巴斯蒂安·斯坦,A_任达华
6,A_史蒂文·元,A_梁朝伟,A_罗宾·威廉姆斯,D_罗伯特·罗德里格兹,A_刘青云
7,A_刘德华,D_费德里科·费里尼,A_连姆·尼森,A_迈克尔·道格拉斯,D_杜琪峰
8,A_李雪健,A_布拉德·皮特,A_布莱恩·科兰斯顿,A_伊万杰琳·莉莉,A_神谷浩史
9,D_郭帆,D_山田洋次,A_西恩·潘,A_保罗·路德,A_梁家辉


# 3

In [20]:
K=5
L=3
prev_theta_tensor = np.ones((L, K, U))
prev_theta_tensor[2]= ottm0.cur_theta
prev_theta_tensor[1]= ottm1.cur_theta
prev_theta_tensor[0]= ottm2.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))
prev_phi_tensor[2]= ottm0.cur_phi
prev_phi_tensor[1]= ottm1.cur_phi
prev_phi_tensor[0]= ottm2.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L


cov_df = cov_dfs[3]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm3=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm3.transform_D()
ottm3._gibbs()
ottm3.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [21]:
T=20
topic2topwords3={}
topic2prob3={}
for j in range(ottm3.K):
    topic2prob3[j]={}
    topic2topwords3[j]=[]
    for w in range((ottm3.V)):
        topic2prob3[j][id2cov_[w]] = ottm3.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob3[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords3[j].append(w1)
        c += 1
        if c == T:
            break
        
print(ottm3.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords3)

[372.70707014 120.05868445 103.00835157  90.42441435 111.8014795 ]


,0,1,2,3,4
0,A_克里斯·帕拉特,D_铃木清顺,A_克里斯·帕拉特,A_克里斯·帕拉特,A_张婧仪
1,D_詹姆斯·古恩,D_香特尔·阿克曼,A_佐伊·索尔达娜,A_克里斯·派恩,D_麦兆辉
2,A_秦昊,D_英格玛·伯格曼,D_詹姆斯·古恩,A_凯伦·吉兰,A_王千源
3,A_佐伊·索尔达娜,D_让-吕克·戈达尔,A_凯伦·吉兰,D_詹姆斯·古恩,A_白百何
4,A_范伟,A_威廉·达福,D_埃里克·侯麦,A_佐伊·索尔达娜,A_刘德华
5,D_辛爽,D_阿基·考里斯马基,A_本尼迪克特·康伯巴奇,A_米歇尔·罗德里格兹,A_黄景瑜
6,A_凯伦·吉兰,A_罗伯特·德尼罗,A_斯嘉丽·约翰逊,D_伍迪·艾伦,A_王丽坤
7,D_孔大山,D_阿涅斯·瓦尔达,D_克日什托夫·基耶斯洛夫斯基,A_尼古拉斯·霍尔特,A_范丞丞
8,A_张婧仪,D_黑泽明,D_宫崎骏,A_清野菜名,D_易小星
9,A_尼古拉斯·凯奇,D_维姆·文德斯,A_休·格兰特,A_尼古拉斯·凯奇,A_张国荣


# 4

In [23]:


prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm1.cur_theta
prev_theta_tensor[1]= ottm2.cur_theta
prev_theta_tensor[0]= ottm3.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm1.cur_phi
prev_phi_tensor[1]= ottm2.cur_phi
prev_phi_tensor[0]= ottm3.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[4]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm4=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm4.transform_D()
ottm4._gibbs()
ottm4.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [24]:

topic2topwords4={}
topic2prob4={}
for j in range(ottm4.K):
    topic2prob4[j]={}
    topic2topwords4[j]=[]
    for w in range((ottm4.V)):
        topic2prob4[j][id2cov_[w]] = ottm4.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob4[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords4[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm4.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords4)

[352.6243341  148.98658191  93.43915903  86.12550227 116.82442268]


,0,1,2,3,4
0,A_米歇尔·罗德里格兹,D_阿基·考里斯马基,A_范·迪塞尔,A_真田广之,A_刘德华
1,A_范·迪塞尔,D_让-吕克·戈达尔,D_林诣彬,A_基努·里维斯,D_王晶
2,A_查理兹·塞隆,D_洪常秀,D_辛爽,A_宋芸桦,D_杜琪峰
3,A_杰森·莫玛,A_金敏喜,A_米歇尔·罗德里格兹,A_杨谨华,A_梁朝伟
4,A_基努·里维斯,D_阿尔弗雷德·希区柯克,A_范伟,A_甄子丹,A_黄秋生
5,D_查德·斯塔赫斯基,D_伍迪·艾伦,A_查理兹·塞隆,D_查德·斯塔赫斯基,A_周星驰
6,A_甄子丹,D_香特尔·阿克曼,A_罗素·克劳,A_安雅·泰勒-乔伊,D_张艺谋
7,A_真田广之,D_王兵,A_秦昊,A_克里斯·帕拉特,A_任达华
8,A_克里斯·帕拉特,D_佩德罗·阿莫多瓦,A_尼古拉斯·凯奇,A_伊藤英明,A_梁家辉
9,A_本·阿弗莱克,A_米歇尔·威廉姆斯,A_杰森·斯坦森,A_尹正,A_刘青云


# 5

In [26]:

prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm2.cur_theta
prev_theta_tensor[1]= ottm3.cur_theta
prev_theta_tensor[0]= ottm4.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm2.cur_phi
prev_phi_tensor[1]= ottm3.cur_phi
prev_phi_tensor[0]= ottm4.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[5]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values


ottm5=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm5.transform_D()
ottm5._gibbs()
ottm5.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [27]:

topic2topwords5={}
topic2prob5={}
for j in range(ottm5.K):
    topic2prob5[j]={}
    topic2topwords5[j]=[]
    for w in range((ottm5.V)):
        topic2prob5[j][id2cov_[w]] = ottm5.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob5[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords5[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm5.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords5)


[349.82246987 142.7345974   99.68899268  85.13168751 120.62225254]


,0,1,2,3,4
0,A_海莉·斯坦菲尔德,A_海莉·斯坦菲尔德,A_奥斯卡·伊萨克,D_文斯·吉里根,A_尹昉
1,A_奥斯卡·伊萨克,D_让-吕克·戈达尔,A_海莉·斯坦菲尔德,A_海莉·斯坦菲尔德,A_海莉·斯坦菲尔德
2,A_尹昉,A_奥斯卡·伊萨克,A_汤姆·克鲁斯,A_基努·里维斯,A_王子文
3,A_基努·里维斯,D_阿基·考里斯马基,A_范·迪塞尔,D_亚当·伯恩斯坦,A_奥斯卡·伊萨克
4,D_查德·斯塔赫斯基,D_香特尔·阿克曼,A_米歇尔·罗德里格兹,A_鲍勃·奥登科克,A_刘德华
5,A_甄子丹,D_埃里克·侯麦,A_柯特妮·考克斯,D_米歇尔·麦克拉伦,A_白客
6,D_宫崎骏,A_罗伯特·德尼罗,A_丹尼尔·克雷格,D_查德·斯塔赫斯基,A_郝蕾
7,A_真田广之,D_是枝裕和,A_汤姆·汉克斯,A_亚伦·保尔,A_周润发
8,A_米歇尔·威廉姆斯,A_北野武,A_汤姆·哈迪,A_奥斯卡·伊萨克,A_刘奕君
9,A_黄子华,A_仲代达矢,D_史蒂文·斯皮尔伯格,A_甄子丹,A_王鸥


# 6

In [29]:

prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm3.cur_theta
prev_theta_tensor[1]= ottm4.cur_theta
prev_theta_tensor[0]= ottm5.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm3.cur_phi
prev_phi_tensor[1]= ottm4.cur_phi
prev_phi_tensor[0]= ottm5.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[6]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values


ottm6=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm6.transform_D()
ottm6._gibbs()
ottm6.fit()

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [30]:

topic2topwords6={}
topic2prob6={}
for j in range(ottm6.K):
    topic2prob6[j]={}
    topic2topwords6[j]=[]
    for w in range((ottm6.V)):
        topic2prob6[j][id2cov_[w]] = ottm6.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob6[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords6[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm6.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords6)

[362.66336053 136.87996743  86.74222009  84.45119503 127.26325693]


,0,1,2,3,4
0,A_本·阿弗莱克,D_让-吕克·戈达尔,A_汤姆·克鲁斯,A_本·阿弗莱克,A_梁家辉
1,A_埃兹拉·米勒,D_迈克·李,D_米歇尔·麦克拉伦,A_埃兹拉·米勒,A_刘德华
2,A_格什菲·法拉哈尼,D_伊丹十三,A_布莱恩·科兰斯顿,A_朱一龙,A_惠英红
3,A_克里斯·海姆斯沃斯,A_让-吕克·戈达尔,D_亚当·伯恩斯坦,D_是枝裕和,A_本·阿弗莱克
4,A_朱一龙,D_维姆·文德斯,A_埃兹拉·米勒,A_安藤樱,A_倪大红
5,A_萨尔玛·海耶克,A_德鲁·巴里摩尔,A_尼古拉斯·凯奇,A_倪妮,A_周润发
6,A_倪妮,D_韦斯·克雷文,A_休·杰克曼,A_菅田将晖,A_埃兹拉·米勒
7,A_倪大红,A_柯特妮·考克斯,A_詹妮弗·劳伦斯,A_永山瑛太,A_吴孟达
8,A_梁家辉,A_伊莎贝尔·于佩尔,A_西蒙·佩吉,D_张艺谋,A_梁朝伟
9,A_惠英红,D_山田洋次,D_史蒂文·斯皮尔伯格,A_黑木华,A_倪妮


# 7

In [32]:

prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm4.cur_theta
prev_theta_tensor[1]= ottm5.cur_theta
prev_theta_tensor[0]= ottm6.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm4.cur_phi
prev_phi_tensor[1]= ottm5.cur_phi
prev_phi_tensor[0]= ottm6.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[7]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values


ottm7=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm7.transform_D()
ottm7._gibbs()
ottm7.fit()

T=10
topic2topwords7={}
topic2prob7={}
for j in range(ottm7.K):
    topic2prob7[j]={}
    topic2topwords7[j]=[]
    for w in range((ottm7.V)):
        topic2prob7[j][id2cov_[w]] = ottm7.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob7[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords7[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm7.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords7)

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

,0,1,2,3,4
0,A_哈里森·福特,D_三宅唱,A_塞缪尔·杰克逊,D_冲田修一,A_郭富城
1,A_安东尼奥·班德拉斯,D_阿彼察邦·韦拉斯哈古,A_汤姆·克鲁斯,A_秦昊,A_王宝强
2,D_詹姆斯·曼高德,A_蒂尔达·斯文顿,D_文斯·吉里根,A_范伟,A_古天乐
3,D_史蒂文·斯皮尔伯格,D_让-吕克·戈达尔,A_罗伯特·德尼罗,A_松田龙平,A_刘青云
4,A_王宝强,D_埃里克·侯麦,D_亚当·伯恩斯坦,A_深津绘里,A_梁家辉
5,A_朱一龙,A_克林特·伊斯特伍德,D_史蒂文·斯皮尔伯格,D_辛爽,D_邱礼涛
6,A_倪妮,D_成濑巳喜男,D_布莱恩·德·帕尔玛,A_马东锡,A_刘德华
7,A_范丞丞,D_侯孝贤,D_金基德,D_新海诚,D_王晶
8,A_张婧仪,A_让-吕克·戈达尔,A_鲍勃·奥登科克,A_朱一龙,A_周星驰
9,D_易小星,A_朱莉·德尔佩,A_柯特妮·考克斯,A_克里斯·帕拉特,A_惠英红


# 8

In [34]:

prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm5.cur_theta
prev_theta_tensor[1]= ottm6.cur_theta
prev_theta_tensor[0]= ottm7.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm5.cur_phi
prev_phi_tensor[1]= ottm6.cur_phi
prev_phi_tensor[0]= ottm7.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[8]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values


ottm8=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm8.transform_D()
ottm8._gibbs()
ottm8.fit()


finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

In [35]:

T=10
topic2topwords8={}
topic2prob8={}
for j in range(ottm8.K):
    topic2prob8[j]={}
    topic2topwords8[j]=[]
    for w in range((ottm8.V)):
        topic2prob8[j][id2cov_[w]] = ottm8.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob8[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords8[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm8.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords8)

[382.26377347 136.93755248  83.45185315  64.50779204 130.83902887]


,0,1,2,3,4
0,A_汤姆·克鲁斯,D_韦斯·安德森,D_阿尔弗雷德·希区柯克,D_黑泽明,A_黄渤
1,A_西蒙·佩吉,A_汤姆·汉克斯,D_克里斯托弗·诺兰,A_松田龙平,A_刘德华
2,A_丽贝卡·弗格森,A_斯嘉丽·约翰逊,D_山田洋次,D_洪常秀,A_李雪健
3,A_海莉·阿特维尔,A_蒂尔达·斯文顿,D_史蒂文·斯皮尔伯格,D_冲田修一,A_西蒙·佩吉
4,A_黄渤,D_让-吕克·戈达尔,A_小罗伯特·唐尼,A_小田切让,A_范丞丞
5,A_李雪健,A_蕾雅·赛杜,A_马特·达蒙,A_染谷将太,A_周迅
6,A_瑞恩·高斯林,D_查理·卓别林,A_哈里森·福特,D_宫崎骏,D_杜琪峰
7,A_玛格特·罗比,A_张国荣,A_柯特妮·考克斯,A_鲍勃·奥登科克,A_刘青云
8,A_本·阿弗莱克,D_阿尔弗雷德·希区柯克,A_艾米莉·布朗特,A_松坂桃李,A_黄秋生
9,A_汤姆·汉克斯,A_克林特·伊斯特伍德,A_詹妮弗·安妮斯顿,D_文斯·吉里根,A_海莉·阿特维尔


# 9

In [37]:


prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm6.cur_theta
prev_theta_tensor[1]= ottm7.cur_theta
prev_theta_tensor[0]= ottm8.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L
prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm6.cur_phi
prev_phi_tensor[1]= ottm7.cur_phi
prev_phi_tensor[0]= ottm8.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[9]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm9=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm9.transform_D()
ottm9._gibbs()
ottm9.fit()

T=10
topic2topwords9={}
topic2prob9={}
for j in range(ottm9.K):
    topic2prob9[j]={}
    topic2topwords9[j]=[]
    for w in range((ottm9.V)):
        topic2prob9[j][id2cov_[w]] = ottm9.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob9[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords9[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm9.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords9)

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

,0,1,2,3,4
0,A_黄渤,D_香特尔·阿克曼,A_马特·达蒙,D_张艺谋,A_黄渤
1,A_瑞恩·高斯林,A_瑞恩·高斯林,D_克里斯托弗·诺兰,D_山田洋次,A_林家栋
2,A_玛格特·罗比,A_玛格特·罗比,A_小罗伯特·唐尼,D_庵野秀明,A_玛格特·罗比
3,A_李雪健,D_查理·卓别林,A_基里安·墨菲,A_巩俐,A_瑞恩·高斯林
4,A_王一博,A_查理·卓别林,A_弗洛伦丝·皮尤,A_菅田将晖,A_刘德华
5,D_大鹏,D_阿尔弗雷德·希区柯克,A_艾米莉·布朗特,D_阿涅斯·瓦尔达,D_郑保瑞
6,D_郑保瑞,A_罗伯特·德尼罗,D_是枝裕和,D_黑泽明,A_梁朝伟
7,A_杰森·斯坦森,A_香特尔·阿克曼,A_佐伊·索尔达娜,A_小田切让,A_黄秋生
8,A_林家栋,D_马丁·斯科塞斯,A_瑞恩·高斯林,A_松坂桃李,A_李雪健
9,A_吴京,D_罗曼·波兰斯基,A_玛格特·罗比,A_滨边美波,A_成龙


# 10

In [39]:

prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm7.cur_theta
prev_theta_tensor[1]= ottm8.cur_theta
prev_theta_tensor[0]= ottm9.cur_theta


alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm7.cur_phi
prev_phi_tensor[1]= ottm8.cur_phi
prev_phi_tensor[0]= ottm9.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[10]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm10=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm10.transform_D()
ottm10._gibbs()
ottm10.fit()

T=10
topic2topwords10={}
topic2prob10={}
for j in range(ottm10.K):
    topic2prob10[j]={}
    topic2topwords10[j]=[]
    for w in range((ottm10.V)):
        topic2prob10[j][id2cov_[w]] = ottm10.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob10[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords10[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm10.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords10)


finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

,0,1,2,3,4
0,A_许光汉,D_让-吕克·戈达尔,D_香特尔·阿克曼,D_山田洋次,A_许光汉
1,A_王传君,A_秦昊,A_马特·达蒙,A_许光汉,A_张艺兴
2,A_张艺兴,A_许光汉,D_克里斯托弗·诺兰,D_岩井俊二,A_古天乐
3,A_秦昊,A_井柏然,A_小罗伯特·唐尼,A_张国荣,A_王传君
4,A_井柏然,A_麦克·辛,A_布拉德·皮特,A_苍井优,D_邱礼涛
5,A_詹妮弗·劳伦斯,A_大卫·田纳特,A_艾米莉·布朗特,A_海莉·斯坦菲尔德,A_黄渤
6,A_乌玛·瑟曼,A_汤姆·克鲁斯,A_基里安·墨菲,A_奥斯卡·伊萨克,A_郭富城
7,A_黄渤,A_罗伯特·德尼罗,D_亚当·伯恩斯坦,A_丁海寅,A_刘青云
8,A_佟丽娅,A_朱丽叶·比诺什,D_文斯·吉里根,A_松山研一,A_成龙
9,A_古天乐,A_乔恩·哈姆,A_詹妮弗·安妮斯顿,A_乌玛·瑟曼,A_吴镇宇


# 11

In [41]:


prev_theta_tensor = np.ones((L, K, U))
prev_theta_tensor[2]= ottm8.cur_theta
prev_theta_tensor[1]= ottm9.cur_theta
prev_theta_tensor[0]= ottm10.cur_theta

alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))
prev_phi_tensor[2]= ottm8.cur_phi
prev_phi_tensor[1]= ottm9.cur_phi
prev_phi_tensor[0]= ottm10.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[11]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm11=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm11.transform_D()
ottm11._gibbs()
ottm11.fit()

T=10
topic2topwords11={}
topic2prob11={}
for j in range(ottm11.K):
    topic2prob11[j]={}
    topic2topwords11[j]=[]
    for w in range((ottm11.V)):
        topic2prob11[j][id2cov_[w]] = ottm11.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob11[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords11[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm11.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords11)

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

,0,1,2,3,4
0,D_克里斯托弗·诺兰,D_克里斯蒂安·佩措尔德,A_阿尔·帕西诺,A_倪妮,A_刘德华
1,A_马特·达蒙,A_斯嘉丽·约翰逊,D_文斯·吉里根,A_朱一龙,A_古天乐
2,A_弗洛伦丝·皮尤,A_伊桑·霍克,A_克里斯蒂安·贝尔,A_克里斯·帕拉特,D_邱礼涛
3,A_艾米莉·布朗特,D_阿尔弗雷德·希区柯克,A_詹妮弗·劳伦斯,D_马丁·斯科塞斯,A_梁家辉
4,A_基里安·墨菲,A_鲁伯特·格林特,A_鲍勃·奥登科克,D_乔尔·科恩,A_倪妮
5,A_小罗伯特·唐尼,D_伍迪·艾伦,D_香特尔·阿克曼,D_詹姆斯·曼高德,A_查理·卓别林
6,D_邱礼涛,A_加里·奥德曼,A_罗宾·威廉姆斯,A_哈里森·福特,A_小罗伯特·唐尼
7,D_陈哲艺,A_艾玛·沃森,D_亚当·伯恩斯坦,A_佐伊·索尔达娜,A_黄秋生
8,A_周冬雨,A_拉尔夫·费因斯,A_杰西卡·查斯坦,D_詹姆斯·古恩,A_哈里森·福特
9,A_刘昊然,D_王家卫,A_罗伯特·帕丁森,D_伊桑·科恩,A_吴孟达


# 12

In [43]:


prev_theta_tensor = np.ones((L, K, U))

prev_theta_tensor[2]= ottm9.cur_theta
prev_theta_tensor[1]= ottm10.cur_theta
prev_theta_tensor[0]= ottm11.cur_theta

alpha_tensor = np.ones((L, K, U))
alpha_tensor *= 1/L

prev_phi_tensor = np.ones((L, K, V))

prev_phi_tensor[2]= ottm9.cur_phi
prev_phi_tensor[1]= ottm10.cur_phi
prev_phi_tensor[0]= ottm11.cur_phi

beta_tensor = np.ones((L, K, V))
beta_tensor *= 1/L

cov_df = cov_dfs[12]
cov_df = cov_df[cov_df.index.isin(valid_user_names)]
cov_df  = cov_df.loc[:, cov_df.columns.isin(valid_cov_names)]
cov_mat = cov_df.values

ottm12=TTM(K=K,cov_mat=cov_mat,n_iters=iters,fix_iters=10,prev_theta_tensor=prev_theta_tensor, prev_phi_tensor=prev_phi_tensor, beta_tensor=beta_tensor, alpha_tensor=alpha_tensor)
ottm12.transform_D()
ottm12._gibbs()
ottm12.fit()

T=10
topic2topwords12={}
topic2prob12={}
for j in range(ottm12.K):
    topic2prob12[j]={}
    topic2topwords12[j]=[]
    for w in range((ottm12.V)):
        topic2prob12[j][id2cov_[w]] = ottm12.cur_phi[j,w]
        
    word2prob = {k: v for k, v in sorted(topic2prob12[j].items(), key=lambda it: -it[1])}
    c = 0
    for w1 in list(word2prob.keys()):
        topic2topwords12[j].append(w1)
        c += 1
        if c == T:
            break
print(ottm12.cur_theta.sum(axis=1))
pd.DataFrame(topic2topwords12)

finish init
finish sampling at iter 0
finish updating alpha at iter 0
finish updating beta at iter 0
finish sampling at iter 1
finish updating alpha at iter 1
finish updating beta at iter 1
finish sampling at iter 2
finish updating alpha at iter 2
finish updating beta at iter 2
finish sampling at iter 3
finish updating alpha at iter 3
finish updating beta at iter 3
finish sampling at iter 4
finish updating alpha at iter 4
finish updating beta at iter 4
finish sampling at iter 5
finish updating alpha at iter 5
finish updating beta at iter 5
finish sampling at iter 6
finish updating alpha at iter 6
finish updating beta at iter 6
finish sampling at iter 7
finish updating alpha at iter 7
finish updating beta at iter 7
finish sampling at iter 8
finish updating alpha at iter 8
finish updating beta at iter 8
finish sampling at iter 9
finish updating alpha at iter 9
finish updating beta at iter 9
finish sampling at iter 10
finish updating alpha at iter 10
finish updating beta at iter 10
finish

,0,1,2,3,4
0,A_齐溪,A_罗伯特·德尼罗,D_伍迪·艾伦,D_大卫·林奇,A_齐溪
1,A_基里安·墨菲,A_汤姆·汉克斯,A_瑞恩·高斯林,A_香川照之,A_倪妮
2,D_克里斯托弗·诺兰,D_香特尔·阿克曼,A_玛格特·罗比,A_黄渤,A_胡歌
3,A_小罗伯特·唐尼,A_瑞恩·高斯林,A_迈克尔·凯恩,A_薛景求,A_林家栋
4,A_马特·达蒙,D_马丁·斯科塞斯,D_雷德利·斯科特,A_布拉德·皮特,A_吴磊
5,A_艾米莉·布朗特,A_汤姆·克鲁斯,A_丹泽尔·华盛顿,A_黑木华,A_白宇
6,A_弗洛伦丝·皮尤,A_梁朝伟,A_克里斯蒂安·贝尔,D_刘伟强,A_张颂文
7,A_林家栋,A_艾米莉·布朗特,D_埃里克·侯麦,A_堺雅人,A_王宝强
8,A_杨子姗,D_乔尔·科恩,A_约瑟夫·高登-莱维特,A_郑伊健,D_魏书钧
9,A_吴磊,A_马特·达蒙,D_克里斯托弗·诺兰,A_舒淇,A_刘德华
